# TMT random groupping parsing code Ver.0.2 Beta
Code update 2021.12.13

## -- input parameters -- 

In [35]:
MetaData = input("\nMetaData 파일 경로와 이름을 입력해 주세요:  \n예제파일 이름:  Meta_data_TMTlabel.txt \n\n ").strip(" ")



MetaData 파일 경로와 이름을 입력해 주세요:  
예제파일 이름:  Meta_data_TMTlabel.txt 

 Meta_data_TMTlabel.txt


In [30]:
NumberOfMultiplex = int(input("\nTMT labeling개수를 입력해 주세요, ex) 6 or 10 or 11:   "))


TMT labeling개수를 입력해 주세요, ex) 6 or 10 or 11:   11


In [32]:
NumberOfGroup = int(input("\n총 그룹의 개수는 몇개인가요? 숫자를 입력해주세요:   " ))


총 그룹의 개수는 몇개인가요? 숫자를 입력해주세요:   5


In [33]:
pooling = input("\nPooling sample을 사용하시나요? \ny 를 선택하면 pooling 샘플을 제외한 상태로 그룹핑이 됩니다. \nn를 선택하면 multiplex 개수를 꽉 채워서 그룹핑이 됩니다 \n\n만약 MetaData에 pooling 샘플 정보가 포함된 경우라면 n 를 선택해 주세요 \n\nSelect y or n :"  ).lower()


Pooling sample을 사용하시나요? 
y 를 선택하면 pooling 샘플을 제외한 상태로 그룹핑이 됩니다. 
n를 선택하면 multiplex 개수를 꽉 채워서 그룹핑이 됩니다 

만약 MetaData에 pooling 샘플 정보가 포함된 경우라면 n 를 선택해 주세요 

Select y or n :y


# --- Code running --- 

In [39]:
import random,sys
Meta = open(MetaData,"r").read().splitlines()   # Data input 하여 한줄당 하나씩 리스트로 반환

plex = NumberOfMultiplex   #pooling 샘플 y면 그룹당 1개씩 풀링 샘플이 들어갈거니깐 plex 개수에서 한자리 비워두기
if pooling == "y":
    plex = plex - 1

def get_groups():         #컬럼 1번 정보를 빼와서 groups에 넣고, 중복 제거하여 group에 집어넣어  conditions 개수 확인
    groups=[]
    for line in Meta[1:len(Meta)]:
        groups.append(line.split()[0]) 
    group = list(set(groups))
    
    return([group,groups])
group = get_groups()[0]
groups = get_groups()[1]
#중복 제거한 그룹 목록 = 0번 (set(column1)), 중복 포함된 그룹 목록 = 1번 (list(column1))

def get_dict():             #Column2번을 Key로, Column 1번을 Value로 하는 딕셔너리 구현
    gorup_dic = {}
    for line in Meta[1:len(Meta)]:
        gorup_dic[str(line.split()[1])] = str(line.split()[0])
    return gorup_dic
dic = get_dict()
# sample label : group label  로 정리된  딕셔너리 구현

def get_Meta():       #Columne 2번을 키로, 해당 샘플의 모든 정보를 value로 하는 딕셔너리 구현 - 최종 출력에 활용
    dic = {}
    for line in Meta[1:len(Meta)]:
        dic[str(line.split()[1])] = line
    return dic
dic_Meta = get_Meta() #sample label : line 으로 정리된 딕셔너리 구현

samplelist_by_groups = []   # 각 그룹별로 리스트 되어있고, 순서가 random으로 섞인 이중 array 구현.
for i in range(0, len(group)):
    sample_set = []
    for key, value in dic.items():
        if value == group[i]:
            sample_set.append(key)
    samplelist_by_groups.append(sample_set)
for i in range(0, len(samplelist_by_groups)):
    a = random.shuffle(samplelist_by_groups[i])

#TMT group 개수에 따라 각각 하나씩 집어넣은 목록 정리.  한 샘플이 한쪽에 쏠리지 않도록 구현
num_of_TMT_set = NumberOfGroup   # input option
random_samples = []
for i in range(0, num_of_TMT_set): 
    sample_set = []
    for i in range(0, len(group)):
        try:
            sample_set.append(samplelist_by_groups[i].pop())
        except:
            pass
    random_samples.append(sample_set)

# 남은것들 한개의 리스트로 묶어주고,  랜덤으로 셔플링 하기
other_samples = sum(samplelist_by_groups,[])

for i in range(0,1000):
    random.shuffle(other_samples)   # 1000 time shuffling

for i in range(0, len(random_samples)):
    while True:
        try:
            if plex > len(random_samples[i]):
                random_samples[i].append(other_samples.pop())
            else:
                break
        except: break
                        
if len(other_samples) != 0:
    sys.exit("포함되지 않는 샘플이 " + str(len(other_samples)) +"개 있습니다. 총 샘플 개수와 TMT-Xplex, 그룹 개수를 다시 확인해 주세요 ")
            
# 남은것들 multiplexing 개수만큼 섞어서 각 그룹별로 넣어주기. 이중 array에서 작업

#--- pooling y or n 에 따라 결과 값에 pooling 샘플 추가 ---#
#pooling == y 이면, NumOfPlex * NumOfGroup - 총 샘플 개수 = pooling 샘플 개수    
if pooling == "y":
    numof_pooling_sample = (NumberOfMultiplex*NumberOfGroup) - len(groups)
    pool = list(range(1,numof_pooling_sample+1))
    pool.reverse()
    for i in range(0, len(random_samples)):
        print("TMT group ",i+1,sep="")
        print(Meta[0])
        for j in range(len(random_samples[i])):
            print(dic_Meta[random_samples[i][j]])
        for p in range(NumberOfMultiplex - len(random_samples[i])):
            pool_num = pool.pop()
            print(f"QC\tpooling\tpool_{pool_num}")
        print('\n')
else:
    for i in range(0, len(random_samples)):
        print("TMT group ",i+1,sep="")
        print(Meta[0])
        for j in range(len(random_samples[i])):
            print( dic_Meta[random_samples[i][j]])
        print('\n')
        

TMT group 1
Group	Group_No	Tube_label	conc.
MT	MT_2	MT3	0.248492534
RM	RM_4	RM9	0.339331345
PT	PT_4	PT5	0.270504699
PM	PM_2	PM4	0.264147395
NL	NL_5	NL7	0.29861811
MT	MT_14	MT20	0.314617326
MT	MT_1	MT2_2nd	0.224511371
PT	PT_8	PT11	0.253454763
PM	PM_13	PM17	0.26855336
MT	MT_3	MT4	0.238497439
QC	pooling	pool_1


TMT group 2
Group	Group_No	Tube_label	conc.
MT	MT_9	MT11	0.305302109
RM	RM_2	RM4	0.302609085
PT	PT_10	PT17	0.285912332
PM	PM_12	PM16	0.252536486
NL	NL_1	NL2	0.267679231
PM	PM_5	PM7	0.282345178
PT	PT_2	PT3	0.264332817
PT	PT_9	PT12	0.256695222
PM	PM_3	PM5	0.238038301
MT	MT_11	MT17	0.270619484
QC	pooling	pool_2


TMT group 3
Group	Group_No	Tube_label	conc.
MT	MT_6	MT10	0.323853076
RM	RM_3	RM8	0.336311626
PT	PT_1	PT1	0.296463691
PM	PM_1	PM3	0.243689238
NL	NL_4	NL6	0.297805788
PM	PM_10	PM13	0.284102266
PT	PT_7	PT10_2nd	0.197483999
PM	PM_11	PM14	0.272314765
MT	MT_13	MT19_2nd	0.248995821
MT	MT_12	MT18	0.302776847
QC	pooling	pool_3


TMT group 4
Group	Group_No	Tube_label	conc.
MT	MT_15	MT

# -- TMT calculation -- 

In [6]:
plex6_tages = ["TMT10-126","TMT10-127","TMT10-128","TMT10-129","TMT10-130","TMT10-131"]
plex10_tages = ["TMT10-126","TMT10-127N","TMT10-127C","TMT10-128N","TMT10-128C","TMT10-129N","TMT10-129C","TMT10-130N","TMT10-130C","TMT10-131"]
plex11_tages = ["TMT10-126","TMT10-127N","TMT10-127C","TMT10-128N","TMT10-128C","TMT10-129N","TMT10-129C","TMT10-130N","TMT10-130C","TMT10-131","TMT10-131C"]

In [7]:
if NumberOfMultiplex == 6:
    tags = plex6_tages
elif NumberOfMultiplex == 10:
    tags = plex10_tages
elif NumberOfMultiplex == 11:
    tags = plex11_tages
    
print(tags)

['TMT10-126', 'TMT10-127N', 'TMT10-127C', 'TMT10-128N', 'TMT10-128C', 'TMT10-129N', 'TMT10-129C', 'TMT10-130N', 'TMT10-130C', 'TMT10-131', 'TMT10-131C']
